In [19]:
%load_ext autoreload
%autoreload 2

from NewsContent import *
from UserContent import *
from preprocessing import *
from PEGenerator import *
import PEGenerator
from models import *
from utils import *
from Encoders import *

import numpy as np
import json


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
data_root_path = "../data/Challenge/"
embedding_path = "../../"
KG_root_path = "../data/Challenge/entity"
popularity_path = "../data/Challenge/popularity"
config = {'title_length':30,
              'body_length':100,
              'max_clicked_news':50,
              'npratio':1,
              'news_encoder_name':"CNN",
              'user_encoder_name':"Att",
             'attrs':['title', 'vert', 'topic'],
             'word_filter':0,
             'data_root_path':data_root_path,
             'embedding_path':embedding_path,
             'KG_root_path':KG_root_path,
            'popularity_path':popularity_path,
            'batch_size': 32,
             'max_entity_num':5,
             'max_topics_num': 5,
             'image_emb_len': 1024}
model_config = {
        'news_encoder':0,
        'popularity_user_modeling':True,
        'rel':True,
        'ctr':True,
        'content':True,
        'rece_emb':True,
        'activity':True

    }

In [22]:
News = NewsContent(config)

TrainUsers = UserContent(News.news_index,config,'train.tsv',2)
ValidUsers = UserContent(News.news_index,config,'val.tsv',1)
TestUsers = UserContent(News.news_index,config,'test.tsv',2)

24724
5071
7607


In [23]:
train_sess,train_buckets, train_user_id, train_label = get_train_input(TrainUsers.session,News.news_index,config)
test_impressions, test_userids = get_test_input(TestUsers.session,News.news_index)
val_impressions, val_userids = get_test_input(ValidUsers.session,News.news_index)

24724
24888
7607
5071


In [24]:
title_word_embedding_matrix, have_word = load_matrix(embedding_path,News.word_dict)
entity_embedding_matrix, have_word2 = load_matrix(embedding_path, News.entity_dict)
if 'topic' in config['attrs']:
    topic_embedding_matrix, have_word3 = load_matrix(embedding_path, News.topic_dict)
else:
    topic_embedding_matrix = np.array([0])

In [25]:
train_loader = DataLoader(TrainDataset(News, TrainUsers, train_sess, train_user_id, train_buckets, train_label), config['batch_size'])
val_user_data = UserDataset(News,ValidUsers)
test_user_data = UserDataset(News,TestUsers)
news_data = NewsDataset(News)

In [26]:
if 'image' in config['attrs']:
    with open(config['embedding_path']+'image_embeddings.pkl', 'rb') as f:
        data = pickle.load(f)

        image_emb_matrix = np.zeros((len(News.news_index)+1, len(data.popitem()[1]),),dtype='float32')

        for key in data:

            if key in News.news_index:
                image_emb_matrix[News.news_index[key]] = data[key]
else:
    image_emb_matrix = np.array([0])


In [27]:
from torch.optim import Adam

model,user_encoder,news_encoder,bias_news_encoder,bias_content_scorer,scaler,time_embedding_layer,activity_gater = \
create_pe_model(config, model_config, News, title_word_embedding_matrix, entity_embedding_matrix, image_emb_matrix, topic_embedding_matrix, device)

model.to(device)
for name, module in model.named_children():
    module.to(device)
    print(f'Moved {name} to {device}')



Moved news_encoder to cuda
Moved time_distributed1 to cuda
Moved pop_aware_user_encoder to cuda
Moved bias_news_encoder to cuda
Moved time_embedding_layer to cuda
Moved bias_scorer to cuda
Moved time_distributed2 to cuda
Moved time_distributed3 to cuda
Moved scaler to cuda
Moved activity_gater to cuda
Moved softmax to cuda


In [29]:
from tqdm import tqdm

# Step 1: Set training/validation variables
colds = [5, 7, 10, 15]
topKs = 10
val_metrics_epoch = []
num_epochs = 10
# Step 2: Create the Adam optimizer
optimizer = Adam(model.parameters(), lr=0.0001)

loss_fn = nn.CrossEntropyLoss()

# Step 3: Iterate over the data for the number of epochs
for epoch in range(num_epochs):

# Step 4: Iterate over each batch of data and compute the scores using the forward pass of the network
    model.train()
    for x, y in tqdm(train_loader):
        optimizer.zero_grad()
        x= [i.to(device) for i in x]
        y = y.to(device)
        out = model(x)
    
        # Step 5: Compute the loss
        loss = loss_fn(out, y)

        # Step 6: Bacward from the scores with the use of the lambda gradient values
        if loss is not None:
            loss.backward()
            
            # Step 7: Update the weights using the optimizer
            optimizer.step()

    # Step 8: Evaluate the model
    model.eval()

    val_metrics = eval_model(model_config, News, user_encoder, val_impressions, val_user_data, val_userids,
               news_encoder, bias_news_encoder, activity_gater, time_embedding_layer, 
               bias_content_scorer, scaler, colds, topKs, ValidUsers, device)
    
    print("epoch: {}, metrics: {}".format(epoch, val_metrics))
    val_metrics_epoch.append(val_metrics)

    # Step 9: Stop when the model has converged
    if epoch > 1:
        if (val_metrics_epoch[-1][0] < val_metrics_epoch[-2][0]):
            break



100%|██████████| 5071/5071 [03:46<00:00, 22.41it/s]


epoch: 0, metrics: ([0.6986895097403562, 0.4824924772806719, 0.5349710738282604, 0.581414289697767], [[0.778392908880197, 0.6692229977305348, 0.7571428571428571, 0.7926170468187276], [0.5676190476190476, 0.3847000057389985, 0.7019230769230769, 0.6428571428571429], [0.6599380048686525, 0.427077964849483, 0.7329899647959225, 0.6802656438775594], [0.6599380048686525, 0.445969518071471, 0.7329899647959225, 0.7311523848929911]], [0.005076429551848894, 0.008786128070507701, 0.011714837427343602, 0.01561978323645814, 0.020305718207395577, 0.023820169435598657, 0.026651740836983948, 0.02821371916062976, 0.029385202903364126, 0.031142428517465664], [-1.0214244983201431e-08, 0.3080921465183146, 0.4065749404804637, 0.4557293337069891, 0.4858921455857411, 0.5013432492520583, 0.5099121891542479, 0.5151462562336674, 0.5187014864408342, 0.521104769373147], [0.999999989785755, 0.6161843033272754, 0.5221088461693963, 0.48880438741389826, 0.47281577817624054, 0.46561045551812524, 0.46069716294120694, 0.

 52%|█████▏    | 408/778 [02:41<02:26,  2.52it/s]


KeyboardInterrupt: 

In [ ]:
test_metrics, test_cold_metrics, test_topics, test_ILADs, test_ILMDs = eval_model(model_config, News, user_encoder, test_impressions, test_user_data, test_userids,
               news_encoder, bias_news_encoder, activity_gater, time_embedding_layer, 
               bias_content_scorer, scaler, colds, topKs, TestUsers, device)

  2%|▏         | 131/7607 [00:03<03:17, 37.91it/s]


KeyboardInterrupt: 

In [ ]:
print("val metrics", val_metrics_epoch)
print("test metrics", test_metrics)
print("cold", test_cold_metrics)
print("diversity", [test_ILADs, test_ILMDs])

results = {"config": config,
           "model_config" : model_config,
            "val metrics": val_metrics_epoch, 
           "test metrics": test_metrics,
           "cold": test_cold_metrics,
           "diversity": [test_ILADs, test_ILMDs]}

with open('results{}.json'.format(time.time()), 'w') as f:
    json.dump(results, f)

val metrics []
test metrics [0.638240975012527, 0.4344798063475666, 0.48158064929225997, 0.5378331465271885]
cold [[nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan]]
diversity [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]


## Finished
## ---